# Assessment 3: RL Gym
### Game Selection: FrozenLake
For this assignment I have chosen the simple game Frozen Lake due to its straighforward mechanics and clear reward structure. The AI is rewarded when it reaches the end of the maze without falling into a hole. As this game has a discrete observation space instead of a continous one, the algorithm used can be much simpler. https://gymnasium.farama.org/environments/toy_text/frozen_lake/

In [ ]:
#Pre-setup installs
%pip install gymnasium[ToyText]
%pip install numpy

In [ ]:
# Setup/imports
import gymnasium
from gymnasium import wrappers
import os
import numpy as np

env = gymnasium.make("FrozenLake-v1", render_mode="rgb_array")  # create the environment used for the game

In [ ]:
# Run this to generate video recordings every 1000 runs or so
env = wrappers.RecordVideo(env, 'recordings')  # wrap environment in recorder to view output
if not os.path.exists('recordings'):  # create directory for storing videos
    os.makedirs('recordings')

### Model Implementation: 
For this game, I chose to use the Q-learning algorithm, primarily as it is one of the simplest algorithms that can be used to show learning and improvement. I modified this by adding an exploration rate that decays over time, meaning the model will rely more and more on its learned behaviours. 
The hyperparameters for this algorithm, shown below, were chosen based on trial and error. I found a higher learning rate would overfit quickly and do worse as the run count increased. With the hyperparameters shown below, the training code can reliably generate a model which can solve the Frozen Lake puzzle ("solving" meaning have a best 100-run average of at least 0.78) in about 6000 runs.

In [ ]:
# Define hyperparameters
number_of_runs = 10000  # takes less than 3 seconds when not recording
learning_rate = 0.1
discount_factor = 0.99
initial_exploration = 1.0
min_exploration = 0.01
exploration_decay = 0.001
report_interval = 500
report = 'Average: %.2f, 100-run average: %.2f, Best average: %.2f (Run %d)'

### Training Process: 
To train the model, I used a table of q-values to track the best move for each step, and a decaying exploration rate which would allow the AI to take random actions and learn their outcomes, which it would do less and less over the course of many runs. In terms of pre-processing, the gymnasium environment provides the map to the AI not as an image, but as an array of letters which represent terrain features. This makes it much easier for the AI to process many runs in a short period of time (10,000 runs took less than 3 seconds on my machine) so training can be done quickly.

In [ ]:
# Reset learned values, rewards and best streak
q_table = np.zeros((env.observation_space.n, env.action_space.n)) # stores learned values
rewards = []
best_streak = 0.0

# Start training
for run in range(number_of_runs):
    observation, info = env.reset()
    done = False
    run_reward = 0
    exploration_rate = max(min_exploration, initial_exploration * np.exp(-exploration_decay * run)) # decrease exploration rate every run
    while not done:
        if np.random.rand() < exploration_rate:
            action = env.action_space.sample()  # Take random actions
        else:
            action = np.argmax(q_table[observation, :])  # Take learned action 

        new_observation, reward, terminated, truncated, _ = env.step(action)

        q_table[observation, action] = (1 - learning_rate) * q_table[observation, action] + learning_rate * \
            (reward + discount_factor * np.max(q_table[new_observation, :]))
        
        run_reward += reward        
        observation = new_observation
        
        if (run + 1) % 100 == 0: # check if last 100 run average was the best so far
            current_streak = np.mean(rewards[-100:])
            if current_streak > best_streak:
                best_streak = current_streak

        if terminated or truncated:
            done = True
            rewards.append(run_reward)
            if ((run + 1) % report_interval == 0): # every 500 runs, print a report showing progress
                print(report % (np.mean(rewards), np.mean(rewards[-100:]), best_streak, run + 1))
env.close()

### Training Evidence:
#### Before training:
<img src="rl-video-episode-27.gif" />


When starting out, the AI tends to wander around randomly, and often falls in a hole very quickly.


#### After 1000 runs:
<img src="rl-video-episode-1000.gif" />


After around 1000 runs, the AI can usually reach the goal about 30% of the time.


#### After 5000 runs:
<img src="rl-video-episode-5000.gif" />


After 5000 runs, the AI still occasionally falls in holes while exploring, but much less often.


#### After 10000 runs:
<img src="rl-video-episode-9000.gif" />


After 6000-8000 runs, the AI tends to get to the goal in more than 80% of runs, though it doesn't usually take the most direct path as there is no incentive for getting to the goal quickly.

### Documentation and Report:
The process of picking a game, picking an algorithm, training the model, refining the model, and then exporting the results took enormous effort, and there were hurdles at every step.


Initially I had planned to create an AI for CartPole, but as CartPole uses a continuous observation space rather than a discrete one like Frozen Lake, this would have limited the types of algorithms I could use to more complex ones that were much harder to configure. I ended up changing to Frozen Lake to get around this limitation.


Picking the algorithm involved a fair amount of research, and I chose the Q-learning algorithm as the other options (DQN, DDPG) were too complex for me to confidently write myself, especially with the timeframe given to complete this project, and the current lack of up-to-date examples for using the Gymnasium environment. 


Training the model required me to search the Gymnasium API to find out what kind of results were being returned from its various methods, to understand why some inputs were resulting in no learning happening at all. Ultimately familiarising myself with the API made it much easier to both design the model and export the results when I was done.


Refining the model was mostly trial and error, and some tweaks resulted in significantly worse performance, like setting the learning rate too high. I also found that setting the discount rate too low resulted in much slower improvement, and a tendency to stall around a 40% success rate. Once I changed the exploration rate from a fixed value to scale down over time, I found that learning happenend much faster and had significantly better results (solving the game in 5000 runs rather than 50,000). 